# Введение в обработку текста на естественном языке

Тарасов С.В.  ДПИ23-1

## Контрольная работа

In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.metrics.distance import edit_distance
import random
import re

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [145]:
preprocessed_descriptions = pd.read_csv('preprocessed_descriptions.csv')
preprocessed_descriptions.head(10)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,preprocessed_description
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,my children and their friends ask for my homem...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,these were so go it surprised even me
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,i think a fondue is a very romantic casual din...
5,mennonite corn fritters,44045,15,41706,2002-10-25,NaN,ok - my heritage has been revealed. :) these a...,NaN,ok my heritage has been revealed these are s...
6,open sesame noodles,107229,28,173674,2004-12-30,8.0,this is a very versatile and widely enjoyed pa...,12.0,this is a very versatile and widely enjoyed pa...
7,say what banana sandwich,95926,5,118163,2004-07-20,4.0,you just have to try it to believe it.,NaN,you just have to try it to believe it
8,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,12.0,this is the recipe that we use at my school ca...,11.0,this is the recipe that we use at my school ca...
9,412 broccoli casserole,306168,40,50969,2008-05-30,6.0,since there are already 411 recipes for brocco...,NaN,since there are already 411 recipes for brocco...


In [146]:
pre_desc = ' '.join(preprocessed_descriptions.preprocessed_description.dropna().tolist())
pre_desc = re.sub(r'[^a-zA-Z\s]', r'', pre_desc)#удалям все числовые значения
words = list(set(word_tokenize(pre_desc)))#создаем список уникальных слов

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [147]:
couple_words = []
while len(couple_words) < 5:
    f_word, s_word = random.choice(words), random.choice(words)
    couple = f_word,s_word
    couple_words.append(couple)


for couple in couple_words:
    print(f'Первое слово-{couple[0]}, второе слово - {couple[1]}\nРасстояние:{edit_distance(couple[0], couple[1])}')
    

Первое слово-stellar, второе слово - tuesday
Расстояние:5
Первое слово-noticeallow, второе слово - krafts
Расстояние:10
Первое слово-restaurantparos, второе слово - ranhofer
Расстояние:12
Первое слово-reminds, второе слово - bartype
Расстояние:7
Первое слово-heaviness, второе слово - benn
Расстояние:7


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [148]:
def func_word(word):
    wo=[]
    for f in words:
        d = edit_distance(word, f)
        if d<3:
            wo.append(f)
    return wo
        
print(func_word('poulos'))

['poupon', 'moulds', 'souls', 'pollo', 'rolos', 'pauls', 'pools', 'pollow', 'paulas', 'poupons', 'polow', 'poulos', 'pours', 'coulis', 'pounds', 'pulls']


### Стемминг, лемматизация

In [149]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.probability import FreqDist

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [150]:
snb_stemmer_en = SnowballStemmer('english')
word_frame = pd.DataFrame(words, columns=['word']).set_index('word')
word_frame['stemmed_word'], word_frame['normalized_word'] = '', ''
for word in words:
    word_frame.loc[[word], 'stemmed_word'] = snb_stemmer_en.stem(word)

In [151]:
lemmatizer = WordNetLemmatizer()

for word in words:
    word_frame.loc[[word], 'normalized_word'] = lemmatizer.lemmatize(word)

In [152]:
word_frame

,stemmed_word,normalized_word
word,,
wwwkochenmachtspassde,wwwkochenmachtspassd,wwwkochenmachtspassde
altered,alter,altered
sierra,sierra,sierra
not,not,not
deliciousto,deliciousto,deliciousto
...,...,...
slice,slice,slice
saltscapes,saltscap,saltscapes
address,address,address


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [153]:
stopwords_en = stopwords.words('english')
words = list(word_tokenize(pre_desc))
without_stopwords = []
for word in words:
    if word not in stopwords_en:
        without_stopwords.append(word)
print(f'Доля стоп слов: {(len(words)-len(without_stopwords))/len(words)*100}')

Доля стоп слов: 46.08283543730754


In [154]:
fdist, sdist = FreqDist(words), FreqDist(without_stopwords)
print(f'ТОП-10 до удаления:{fdist.most_common(10)}\nТОП-10 после:{sdist.most_common(10)}')

ТОП-10 до удаления:[('the', 40078), ('a', 34971), ('and', 30245), ('this', 26865), ('i', 24840), ('to', 23473), ('is', 20285), ('it', 19758), ('of', 18364), ('for', 15940)]
ТОП-10 после:[('recipe', 14962), ('make', 6326), ('time', 5140), ('use', 4622), ('great', 4431), ('like', 4168), ('easy', 4153), ('one', 3876), ('made', 3810), ('good', 3793)]


### Векторное представление текста

In [155]:
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from scipy.spatial import distance

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [156]:
recipes = preprocessed_descriptions.sample(n = 5)
list_recipes = recipes.preprocessed_description.dropna().tolist()

tv = TfidfVectorizer()
recipes_tv = tv.fit_transform(list_recipes)
recipes_tv = recipes_tv.toarray()
recipes_tv

array([[0.        , 0.        , 0.        , 0.        , 0.16951841,
        0.        , 0.        , 0.14071559, 0.        , 0.        ,
        0.2101137 , 0.        , 0.        , 0.        , 0.        ,
        0.14071559, 0.        , 0.        , 0.        , 0.16951841,
        0.16951841, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.2101137 ,
        0.2101137 , 0.2101137 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.11837439,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2101137 , 0.        , 0.        , 0.  

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [157]:
frame = pd.DataFrame(recipes.name, columns = ['name'])
for i in range(5):
    frame[recipes.iloc[i, 0]] = ''
    
for a in range(5):
    for i in range(1, 6):
        frame.iloc[a, i] = distance.cosine(recipes_tv[a], recipes_tv[i-1])
        
frame.set_index('name')

,dube s brandy apple pie,biscuit mix pizza dough,goat cheese gourmet vegetarian pizza,spiced orange pecans,double pea mash
name,,,,,
dube s brandy apple pie,0.0,0.887487,1.0,0.895327,0.920355
biscuit mix pizza dough,0.887487,0.0,0.939517,0.78068,0.819585
goat cheese gourmet vegetarian pizza,1.0,0.939517,0.0,0.947476,0.982678
spiced orange pecans,0.895327,0.78068,0.947476,0.0,0.6956
double pea mash,0.920355,0.819585,0.982678,0.6956,0.0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).